# 10. 얼굴인식(Haar Cascade)

In [ ]:
import cv2 as cv
FACE = "../images/face1.jpg"

FACE_CASCADE = "../cascade/haarcascade_frontalface_default.xml"
EYE_CASCADE = "../cascade/haarcascade_eye.xml"

In [ ]:
face_cascade = cv.CascadeClassifier(FACE_CASCADE)

img = cv.imread(FACE)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(
    gray, scaleFactor=1.1, minNeighbors=10, minSize=(10,10)
)

if len(faces):
    for face in faces:
        x, y, width, height = face
        cv.rectangle(img, (x,y), (x+width, y+height), (0,255,0), 2, cv.LINE_AA)

cv.imshow("img", img)

cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

In [ ]:
eye_cascade = cv.CascadeClassifier(EYE_CASCADE)

img = cv.imread(FACE)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

eyes = eye_cascade.detectMultiScale(
    gray, scaleFactor=1.1, minNeighbors=15, minSize=(10,10)
)

if len(eyes):
    for eye in eyes:
        x, y, width, height = eye
        cv.rectangle(img, (x,y), (x+width, y+height), (0,255,0), 2, cv.LINE_AA)

cv.imshow("img", img)

cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)